# Preprocesamiento de Datos
## Dataset: YouToxic English 1000

Este notebook contiene todas las técnicas de preprocesamiento aplicadas al texto de los comentarios antes de entrenar los modelos de Machine Learning.


## 1. Importación de librerías

Importamos todas las librerías necesarias para el preprocesamiento de texto.


In [ ]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np
import re

# Librerías para NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import spacy

# Descargar recursos de NLTK (solo la primera vez)
print("Descargando recursos de NLTK...")
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab', quiet=True)
    print("✅ punkt_tab descargado")

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', quiet=True)
    print("✅ stopwords descargado")

print("✅ Recursos de NLTK listos")

# Cargar modelo de spaCy para lematización (inglés)
try:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    print("✅ Modelo de spaCy cargado")
except OSError:
    print("⚠️  Modelo de spaCy no encontrado. Ejecuta: python -m spacy download en_core_web_sm")
    nlp = None

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 150)


ModuleNotFoundError: No module named 'nltk'

## 2. Carga de datos

Cargamos el dataset original.


In [ ]:
# Cargar el dataset
df = pd.read_csv('../data/raw/youtoxic_english_1000.csv')

print(f"Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")
print(f"\nPrimeras filas:")
df.head()


## 3. Verificación de datos

Verificamos valores nulos y duplicados antes de empezar el preprocesamiento.


In [ ]:
# Verificar valores nulos
print("Valores nulos por columna:")
print(df.isnull().sum())
print(f"\nTotal de valores nulos: {df.isnull().sum().sum()}")

# Verificar duplicados
print(f"\nNúmero de filas duplicadas: {df.duplicated().sum()}")
print(f"Número de comentarios duplicados (por texto): {df['Text'].duplicated().sum()}")


## 4. Funciones de preprocesamiento

Definimos todas las funciones necesarias para limpiar y preprocesar el texto.


In [ ]:
def clean_text(text):
    """
    Limpia el texto eliminando URLs, emails y caracteres especiales.
    
    Args:
        text (str): Texto a limpiar
        
    Returns:
        str: Texto limpio
    """
    if pd.isna(text):
        return ""
    
    # Convertir a string si no lo es
    text = str(text)
    
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Eliminar emails
    text = re.sub(r'\S+@\S+', '', text)
    
    # Eliminar caracteres especiales pero mantener espacios y puntuación básica
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Eliminar espacios múltiples
    text = re.sub(r'\s+', ' ', text)
    
    # Convertir a minúsculas
    text = text.lower().strip()
    
    return text


# Ejemplo de uso
ejemplo = "Check out this website: https://example.com and email me at test@email.com!!!"
print("Texto original:", ejemplo)
print("Texto limpio:", clean_text(ejemplo))


In [ ]:
def normalize_text(text):
    """
    Normaliza el texto usando expresiones regulares.
    - Normaliza contracciones (don't -> do not)
    - Normaliza repeticiones de caracteres (sooo -> so)
    - Normaliza espacios
    
    Args:
        text (str): Texto a normalizar
        
    Returns:
        str: Texto normalizado
    """
    if pd.isna(text) or text == "":
        return ""
    
    text = str(text)
    
    # Normalizar contracciones comunes
    contractions = {
        "don't": "do not",
        "won't": "will not",
        "can't": "cannot",
        "n't": " not",
        "'re": " are",
        "'ve": " have",
        "'ll": " will",
        "'m": " am",
        "'d": " would"
    }
    
    for contraction, expansion in contractions.items():
        text = re.sub(contraction, expansion, text, flags=re.IGNORECASE)
    
    # Normalizar repeticiones de caracteres (máximo 2 repeticiones)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    # Eliminar espacios múltiples
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()


# Ejemplo de uso
ejemplo = "I don't think sooo this is gooood!!!"
print("Texto original:", ejemplo)
print("Texto normalizado:", normalize_text(ejemplo))


In [ ]:
def tokenize_text(text):
    """
    Tokeniza el texto en palabras individuales.
    
    Args:
        text (str): Texto a tokenizar
        
    Returns:
        list: Lista de tokens (palabras)
    """
    if pd.isna(text) or text == "":
        return []
    
    # Usar NLTK para tokenización
    tokens = word_tokenize(str(text))
    
    # Filtrar tokens vacíos
    tokens = [token for token in tokens if token.strip()]
    
    return tokens


# Ejemplo de uso
ejemplo = "This is a sample text for tokenization!"
print("Texto original:", ejemplo)
print("Tokens:", tokenize_text(ejemplo))


In [ ]:
def remove_stopwords(tokens, language='english'):
    """
    Elimina las stopwords (palabras comunes sin significado) de una lista de tokens.
    
    Args:
        tokens (list): Lista de tokens
        language (str): Idioma de las stopwords ('english' por defecto)
        
    Returns:
        list: Lista de tokens sin stopwords
    """
    if not tokens:
        return []
    
    # Obtener stopwords
    stop_words = set(stopwords.words(language))
    
    # Filtrar stopwords
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    
    return filtered_tokens


# Ejemplo de uso
ejemplo_tokens = tokenize_text("This is a sample text for removing stopwords")
print("Tokens originales:", ejemplo_tokens)
print("Tokens sin stopwords:", remove_stopwords(ejemplo_tokens))


In [ ]:
def stem_text(tokens):
    """
    Aplica stemming a los tokens (reduce palabras a su raíz).
    Usa Porter Stemmer de NLTK.
    
    Args:
        tokens (list): Lista de tokens
        
    Returns:
        list: Lista de tokens con stemming aplicado
    """
    if not tokens:
        return []
    
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    return stemmed_tokens


# Ejemplo de uso
ejemplo_tokens = tokenize_text("running runs ran")
print("Tokens originales:", ejemplo_tokens)
print("Tokens con stemming:", stem_text(ejemplo_tokens))


In [ ]:
def lemmatize_text(text, nlp_model=None):
    """
    Aplica lematización al texto (reduce palabras a su forma base/lema).
    Usa spaCy para lematización (más preciso que stemming).
    
    Args:
        text (str): Texto a lematizar
        nlp_model: Modelo de spaCy cargado
        
    Returns:
        str: Texto lematizado
    """
    if pd.isna(text) or text == "":
        return ""
    
    if nlp_model is None:
        return str(text)
    
    # Procesar texto con spaCy
    doc = nlp_model(text)
    
    # Extraer lemas
    lemmas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    
    return ' '.join(lemmas)


# Ejemplo de uso (si spaCy está disponible)
if nlp is not None:
    ejemplo = "running runs ran better"
    print("Texto original:", ejemplo)
    print("Texto lematizado:", lemmatize_text(ejemplo, nlp))
else:
    print("⚠️  spaCy no está disponible. Instala el modelo con: python -m spacy download en_core_web_sm")


## 5. Pipeline completo de preprocesamiento

Aplicamos todas las funciones de preprocesamiento en secuencia.


In [ ]:
def preprocess_text(text, use_lemmatization=True, use_stemming=False):
    """
    Pipeline completo de preprocesamiento de texto.
    
    Args:
        text (str): Texto original
        use_lemmatization (bool): Si True, usa lematización (requiere spaCy)
        use_stemming (bool): Si True, usa stemming (solo si no se usa lematización)
        
    Returns:
        str: Texto preprocesado
    """
    # 1. Limpieza básica
    text = clean_text(text)
    
    # 2. Normalización
    text = normalize_text(text)
    
    # 3. Tokenización
    tokens = tokenize_text(text)
    
    # 4. Eliminación de stopwords
    tokens = remove_stopwords(tokens)
    
    # 5. Stemming o Lematización
    if use_lemmatization and nlp is not None:
        # Para lematización, necesitamos el texto completo
        text_for_lemma = ' '.join(tokens)
        processed_text = lemmatize_text(text_for_lemma, nlp)
    elif use_stemming:
        # Aplicar stemming
        tokens = stem_text(tokens)
        processed_text = ' '.join(tokens)
    else:
        # Sin stemming ni lematización
        processed_text = ' '.join(tokens)
    
    return processed_text.strip()


# Ejemplo completo
ejemplo = "Check out https://example.com! I don't think sooo this is gooood!!!"
print("Texto original:")
print(ejemplo)
print("\n" + "="*60)
print("Texto preprocesado (con lematización):")
print(preprocess_text(ejemplo, use_lemmatization=True, use_stemming=False))
print("\n" + "="*60)
print("Texto preprocesado (con stemming):")
print(preprocess_text(ejemplo, use_lemmatization=False, use_stemming=True))


## 6. Aplicar preprocesamiento al dataset

Aplicamos el preprocesamiento a todos los comentarios del dataset.


In [ ]:
# Crear una copia del dataframe para trabajar
df_processed = df.copy()

# Aplicar preprocesamiento con lematización (más preciso)
print("Aplicando preprocesamiento con lematización...")
print("Esto puede tardar unos minutos...")

if nlp is not None:
    df_processed['Text_processed'] = df_processed['Text'].apply(
        lambda x: preprocess_text(x, use_lemmatization=True, use_stemming=False)
    )
else:
    print("⚠️  Usando stemming porque spaCy no está disponible")
    df_processed['Text_processed'] = df_processed['Text'].apply(
        lambda x: preprocess_text(x, use_lemmatization=False, use_stemming=True)
    )

print("✅ Preprocesamiento completado!")


In [ ]:
# Ver ejemplos de texto original vs preprocesado
print("="*80)
print("EJEMPLOS: Texto Original vs Preprocesado")
print("="*80)

for i in range(5):
    print(f"\nEjemplo {i+1}:")
    print(f"Original: {df_processed.iloc[i]['Text'][:150]}...")
    print(f"Preprocesado: {df_processed.iloc[i]['Text_processed'][:150]}...")
    print("-" * 80)


## 7. Manejo de valores nulos y duplicados

Verificamos y manejamos cualquier problema restante.


In [ ]:
# Verificar si hay textos preprocesados vacíos
empty_processed = df_processed['Text_processed'].str.strip() == ''
print(f"Textos preprocesados vacíos: {empty_processed.sum()}")

# Mostrar algunos ejemplos si los hay
if empty_processed.sum() > 0:
    print("\nEjemplos de textos que quedaron vacíos después del preprocesamiento:")
    empty_examples = df_processed[empty_processed][['Text', 'Text_processed']].head(5)
    print(empty_examples)


In [ ]:
# Eliminar filas con texto preprocesado vacío (si las hay)
if empty_processed.sum() > 0:
    print(f"Eliminando {empty_processed.sum()} filas con texto vacío...")
    df_processed = df_processed[~empty_processed].copy()
    print(f"Dataset después de eliminar vacíos: {df_processed.shape[0]} filas")

# Verificar duplicados en el texto preprocesado
duplicated_processed = df_processed['Text_processed'].duplicated()
print(f"\nTextos preprocesados duplicados: {duplicated_processed.sum()}")

# Decisión: mantener duplicados por ahora (pueden ser comentarios legítimamente iguales)
# Si queremos eliminarlos, descomentar la siguiente línea:
# df_processed = df_processed[~duplicated_processed].copy()


## 8. Guardar datos preprocesados

Guardamos el dataset preprocesado para usarlo en el modelado.


In [ ]:
# Crear directorio processed si no existe
import os
os.makedirs('../data/processed', exist_ok=True)

# Guardar dataset preprocesado
output_path = '../data/processed/youtoxic_english_1000_processed.csv'
df_processed.to_csv(output_path, index=False)

print(f"✅ Dataset preprocesado guardado en: {output_path}")
print(f"Shape del dataset guardado: {df_processed.shape}")
print(f"\nColumnas guardadas:")
print(df_processed.columns.tolist())


## 9. Resumen del preprocesamiento

Resumen de las transformaciones aplicadas.


In [ ]:
print("="*80)
print("RESUMEN DEL PREPROCESAMIENTO")
print("="*80)

print(f"\n1. DATASET:")
print(f"   - Filas originales: {len(df)}")
print(f"   - Filas después del preprocesamiento: {len(df_processed)}")
print(f"   - Filas eliminadas: {len(df) - len(df_processed)}")

print(f"\n2. TRANSFORMACIONES APLICADAS:")
print(f"   ✅ Limpieza de URLs y emails")
print(f"   ✅ Eliminación de caracteres especiales")
print(f"   ✅ Normalización de texto (contracciones, repeticiones)")
print(f"   ✅ Tokenización")
print(f"   ✅ Eliminación de stopwords")
if nlp is not None:
    print(f"   ✅ Lematización (usando spaCy)")
else:
    print(f"   ✅ Stemming (usando NLTK)")

print(f"\n3. ESTADÍSTICAS DEL TEXTO PREPROCESADO:")
df_processed['text_length_original'] = df_processed['Text'].str.len()
df_processed['text_length_processed'] = df_processed['Text_processed'].str.len()
df_processed['word_count_processed'] = df_processed['Text_processed'].str.split().str.len()

print(f"   - Longitud promedio original: {df_processed['text_length_original'].mean():.1f} caracteres")
print(f"   - Longitud promedio preprocesado: {df_processed['text_length_processed'].mean():.1f} caracteres")
print(f"   - Palabras promedio preprocesado: {df_processed['word_count_processed'].mean():.1f} palabras")

print(f"\n4. ARCHIVO GUARDADO:")
print(f"   - Ruta: ../data/processed/youtoxic_english_1000_processed.csv")

print("\n" + "="*80)
